# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [5]:
full_data_rows_list

[['Hoobastank',
  'Logged In',
  'Cierra',
  'F',
  '0',
  'Finley',
  '241.3971',
  'free',
  'Richmond, VA',
  'PUT',
  'NextSong',
  '1.54101E+12',
  '132',
  'Say The Same',
  '200',
  '1.54181E+12',
  '96'],
 ['Mark Knopfler',
  'Logged In',
  'Cierra',
  'F',
  '1',
  'Finley',
  '249.3122',
  'free',
  'Richmond, VA',
  'PUT',
  'NextSong',
  '1.54101E+12',
  '132',
  'Why Aye Man',
  '200',
  '1.54181E+12',
  '96'],
 ['Mogwai',
  'Logged In',
  'Cierra',
  'F',
  '2',
  'Finley',
  '341.28934',
  'free',
  'Richmond, VA',
  'PUT',
  'NextSong',
  '1.54101E+12',
  '132',
  "We're No Here",
  '200',
  '1.54181E+12',
  '96'],
 ['The Casualties',
  'Logged In',
  'Cierra',
  'F',
  '3',
  'Finley',
  '181.49832',
  'free',
  'Richmond, VA',
  'PUT',
  'NextSong',
  '1.54101E+12',
  '132',
  'Punx Unite',
  '200',
  '1.54181E+12',
  '96'],
 ['',
  'Logged In',
  'Cecilia',
  'F',
  '0',
  'Owens',
  '',
  'free',
  'Atlanta-Sandy Springs-Roswell, GA',
  'GET',
  'Home',
  '1.54103E+

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_key_space 
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}
""")

#### Set Keyspace

In [6]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify_key_space')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [20]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query =  "CREATE TABLE IF NOT EXISTS sessions_1 (sessionId int, itemInSession int, artistName text, songName text, songLength float, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query) 

In [21]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        #query = "<ENTER INSERT STATEMENT HERE>"
        #query = query + "<ASSIGN VALUES HERE>"
        query = "INSERT INTO sessions_1 (sessionId, itemInSession, artistName, songName, songLength)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        #session.execute(query, (line[#], line[#]))
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [22]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select artistName, songName, songLength from sessions_1 WHERE sessionId = 338 and itemInSession = 4"
rows = session.execute(query)
for row in rows:
    print (row)

Row(artistname='Faithless', songname='Music Matters (Mark Knight Dub)', songlength=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [24]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query =  "CREATE TABLE IF NOT EXISTS sessions_2 (userId int, sessionId int, itemInSession int, artistName text, songName text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))"
session.execute(query)

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO sessions_2 (userId, sessionId, itemInSession, artistName, songName, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

query = "select artistName, songName, firstName, lastName from sessions_2 WHERE userId = 10 and sessionId = 182 ORDER BY itemInSession"
rows = session.execute(query)
for row in rows:
    print (row)
                    

Row(artistname='Down To The Bone', songname="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artistname='Three Drives', songname='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artistname='Sebastien Tellier', songname='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artistname='Lonnie Gordon', songname='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [32]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query =  "CREATE TABLE IF NOT EXISTS sessions_3 (firstName text, lastName text, songName text, userId int, PRIMARY KEY(songName, userId))"
session.execute(query)    

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO sessions_3 (firstName, lastName, songName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[1], line[4], line[9], int(line[10])))

query = "select songName, firstName, lastName, songName, userId from sessions_3 WHERE songName='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print (row)   

Row(songname='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch', songname_='All Hands Against His Own', userid=29)
Row(songname='All Hands Against His Own', firstname='Tegan', lastname='Levine', songname_='All Hands Against His Own', userid=80)
Row(songname='All Hands Against His Own', firstname='Sara', lastname='Johnson', songname_='All Hands Against His Own', userid=95)


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [34]:
## Drop the table before closing out the sessions
query = "DROP TABLE sessions_1"
session.execute(query)

query = "DROP TABLE sessions_2"
session.execute(query)

query = "DROP TABLE sessions_3"
session.execute(query)


### Close the session and cluster connection¶

In [35]:
session.shutdown()
cluster.shutdown()